<a href="https://colab.research.google.com/github/MEDHANGSHI/NASA-PACE/blob/main/Copy_of_oceantraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets torch fastapi uvicorn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

In [ ]:
pip install transformers datasets torch


In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model_name = "bert-base-uncased"  # You can also choose 'gpt2' for generation tasks
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
{
  "data": [
    {
      "title": "Ocean Literacy",
      "paragraphs": [
        {
          "context": "Phytoplankton are important microorganisms that live in oceans and contribute to marine ecosystems.",
          "qas": [
            {
              "question": "What are phytoplankton?",
              "id": "001",
              "answers": [
                {
                  "text": "microorganisms that live in oceans",
                  "answer_start": 18
                }
              ]
            }
          ]
        }
      ]
    }
  ]
}


{'data': [{'title': 'Ocean Literacy',
   'paragraphs': [{'context': 'Phytoplankton are important microorganisms that live in oceans and contribute to marine ecosystems.',
     'qas': [{'question': 'What are phytoplankton?',
       'id': '001',
       'answers': [{'text': 'microorganisms that live in oceans',
         'answer_start': 18}]}]}]}]}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
    # Tokenize the context and question, with truncation and padding
    inputs = tokenizer(
        examples['question'], examples['context'],
        truncation=True, padding='max_length', max_length=512
    )
    # Identify where the answer starts and ends in the tokenized context
    answers = examples['answers']
    start_positions = [tokenizer(text)['input_ids'].index(answer['answer_start']) for answer in answers]
    end_positions = [start + len(tokenizer(answer['text'])) for start, answer in zip(start_positions, answers)]
    inputs.update({'start_positions': start_positions, 'end_positions': end_positions})
    return inputs


In [ ]:
from transformers import Trainer, TrainingArguments

# Load dataset (Assume it is in Hugging Face dataset format)
from datasets import load_dataset
dataset = load_dataset('json', data_files={'train': 'train.json', 'validation': 'val.json'})

# Preprocess the dataset
train_dataset = dataset['train'].map(preprocess_data, batched=True)
val_dataset = dataset['validation'].map(preprocess_data, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()


FileNotFoundError: Unable to find '/content/train.json'

In [ ]:
results = trainer.evaluate()
print(f"Evaluation Results: {results}")


NameError: name 'trainer' is not defined

In [ ]:
model.save_pretrained("./ocean-assistant")
tokenizer.save_pretrained("./ocean-assistant")


('./ocean-assistant/tokenizer_config.json',
 './ocean-assistant/special_tokens_map.json',
 './ocean-assistant/vocab.txt',
 './ocean-assistant/added_tokens.json',
 './ocean-assistant/tokenizer.json')

In [ ]:
from fastapi import FastAPI
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

app = FastAPI()

model_name = "path/to/saved/model"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

@app.post("/ask")
def ask_question(question: str, context: str):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    start_scores, end_scores = model(**inputs)
    start_idx = torch.argmax(start_scores)
    end_idx = torch.argmax(end_scores)
    answer = tokenizer.decode(inputs['input_ids'][0][start_idx:end_idx+1])
    return {"answer": answer}


OSError: Incorrect path_or_model_id: 'path/to/saved/model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.